# Inference Notebook for 3D models

## Importing all necessary classes and functions

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys

sys.path.insert(0, "../../src")

import h5py
import torch
import zarr as z
import numpy as np

from juart.conopt.functional.fourier import nonuniform_fourier_transform_adjoint
from juart.dl.checkpoint.manager import CheckpointManager
from juart.dl.model.unrollnet import ExponentialMovingAverageModel, UnrolledNet
from juart.dl.operation.modules import inference
from juart.dl.model.resnet import ResNet
from juart.conopt.tfs.fourier import nonuniform_transfer_function
from juart.conopt.functional.fourier import nonuniform_fourier_transform_adjoint, fourier_transform_forward, fourier_transform_adjoint
from juart.vis.interactive import InteractiveFigure3D, InteractiveMultiPlotter3D

In [ ]:
# To improve performance, manually limit the number of threads
# torch.set_num_threads(16)
# torch.set_num_interop_threads(16)

## Defining all important variables for the inference

In [ ]:
# dataset options
kspace_cutoff = False
nX_cutoff, nY_cutoff, nZ_cutoff = 64, 64, 64 

# device options
device = "cuda:1" # device on which the reconstructions will run

# CheckpointManager options
directory = "corr_modelDD_01i_25P_50DC_epoch_7" # directory of the dl-qrage model
root_dir = "/home/jovyan/models" # the path to the model directory
backend = "local" # the backend of the model directory

# model options
shape = (156,156,156,2,1) # number of pixels in every direction // number of measurements at T1/T2 decay
nX, nY, nZ, nTI, nTE = shape # ordered structure of the parameters above that will be passed to the model
features = 32 # number of hidden_inputs of the dl-qrage model
cg_iter = 50 # number of cg iterations in the dl-qrage model reconstruction

# display options
vmax = 2 # sets the brightness normalization of the display between 0 and vmax
iTI, iTE = 1, 0 # sets the number of measurement that should be displayed

# saving
saving = True

dtype = torch.complex64

## Loading the dataset that should be reconstructed
#### Loading Data

In [ ]:
data_path = "/home/jovyan/juart/examples/data/3DLiss_vd_preproc.h5"
with h5py.File(data_path, "r") as f:
    
    k = torch.from_numpy(f['k'][:])[...,None]
    C = torch.from_numpy(f['coilsens'][:])
    d = torch.from_numpy(f['d'][:])[...,None]

    print(f"Coilsensitivity shape {C.shape}")
    print(f"Trajectory shape {k.shape}")
    print(f"Signal shape {d.shape}")

k /= (2*k.max())

#### Shaping data

In [ ]:
if kspace_cutoff == True:
    nX, nY ,nZ = nX_cutoff, nY_cutoff, nZ_cutoff
    shape = (nX, nY, nZ, nTI, nTE)
    
    mask = torch.linalg.norm(ktraj,dim=0) <= nX_cutoff//2
    
    ktraj = torch.stack(
        [ktraj[:, mask[:, echo], echo] for echo in range(ktraj.shape[2])],
        dim=-1
    )
    
    d = torch.stack(
        [d[:, mask[:, echo], echo] for echo in range(d.shape[2])],
        dim=-1
    )

    coilsens_ksp = fourier_transform_forward(coilsens, axes=(1,2,3))
    low_lim, up_lim = int(156/2 - nX/2), int(156/2 + nX/2)
    coilsens_ksp = coilsens_ksp[:, low_lim:up_lim, low_lim:up_lim, low_lim:up_lim]
    coilsens = fourier_transform_adjoint(coilsens_ksp, axes=(1,2,3))

In [ ]:
kspace_mask_source = torch.randint(0,2,(1, k.shape[1], 1,1))
kspace_mask_target = 1 - kspace_mask_source

k_scaled = k / (2 * k.max())

AHd = nonuniform_fourier_transform_adjoint(k_scaled,d,(nX,nY,nZ))
AHd = torch.sum(torch.conj(C[...,None,None]) * AHd, dim=0)

In [ ]:
data = {
    "images_regridded": AHd,
    "kspace_trajectory": k_scaled,
    "sensitivity_maps": C,
    "kspace_mask_source": kspace_mask_source,
    "kspace_mask_target": kspace_mask_target,
    "kspace_data": d,
}

## Checkpoint Manager
#### Initializing the CheckpointManager and loading the current state

In [ ]:
checkpoint_manager = CheckpointManager(
    directory=directory,
    root_dir=root_dir,
    backend=backend,
)

checkpoint = checkpoint_manager.load(
    ["averaged_model_state", "iteration"], map_location="cuda:3"
)

## Model
#### Initializing the model and using the CheckpointManager to check for save files of the model and load them.

In [ ]:
model = UnrolledNet(
    shape,
    features=features,
    CG_Iter=cg_iter,
    num_unroll_blocks=10,
    activation="ReLU",
    kernel_size = (3,3,3),
    axes = (1,2,3),
    disable_progress_bar=False,
    ResNetCheckpoints = True,
    ConvLayerCheckpoints = False,
    device=device,
    dtype = dtype
)

model = ExponentialMovingAverageModel(model, 0.9)
model.load_state_dict(checkpoint["averaged_model_state"])

iteration = checkpoint["iteration"]
print(f"Loaded averaged at iteration {iteration}.")

## Image reconstruction

In [ ]:
images = inference(data, model, device=device)

### Displaying the reconstructed image in an interactive Figure

In [ ]:
InteractiveFigure3D(
    torch.abs(images[...,0,0]).cpu().numpy(),
    vmin=0,
    vmax=torch.abs(images[...,0,0]).cpu().max(),
    cmap="gray",
).interactive

In [ ]:
if saving:
    torch.save(images,f'/home/jovyan/images/{directory}')